In [94]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time
websiteTF='https://www.cinemarkca.com/index'
websiteCinemas='https://cinemas.com.ni/cartelera/'
path='/home/bahar/Descargas/chromedriver'


In [135]:
driver = webdriver.Chrome(path)
driver.get(websiteCinemas)
#def CerrarModal():
#    modal= driver.find_element('id','modal-theatre-select')
#    ventana= modal.find_element(By.CLASS_NAME,'change-theatre-button')
#    botonCerrar=ventana.find_element('xpath','//button[@class="btn-primary next"]')
#    botonCerrar.click()

def elegir():
#    for i in 5:
        cine=Select(driver.find_element('id','cnm_mov'))
        options = cine.options
        for index in range(0, len(options) - 1):
            cine.select_by_index(index)
            time.sleep(2)
        time.sleep(2)


time.sleep(3)

link=[]
cartelera=driver.find_elements('xpath','//div[@data-total="11"]/a[@class="btn-red"]')
for movie in cartelera:
    link.append(movie.get_attribute("href"))
    
for i in link:
    #driver.execute_script("window.open('');")
    #driver.switch_to.window(driver.window_handles[1])
    driver.get(i)
    elegir()
    driver.back()
    #driver.close()
    #driver.switch_to.window(driver.window_handles[0])

time.sleep(2)

driver.quit()

/tmp/ipykernel_3438/3908198718.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
